In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

Matplotlib is building the font cache; this may take a moment.


In [2]:
# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

In [3]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [4]:
bucket_name = 'c3-capstone'
s3 = boto3.resource('s3')
try:g
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [5]:
bucket='c3-capstone'
subfolder ='input'
from sagemaker import get_execution_role
role=get_execution_role()
conn=boto3.client('s3')
contents=conn.list_objects(Bucket=bucket,Prefix=subfolder)['Contents']
for f in contents:
    print(f['Key'])

input/
input/Capstone.csv


In [31]:
try:
  urllib.request.urlretrieve ("https://c3-capstone.s3.amazonaws.com/input/Capstone.csv", "Capstone.csv")
  print('Success: downloaded Capstone.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./Capstone.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded Capstone.csv.
Success: Data loaded into dataframe.


In [48]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(model_data, test_size = 0.3, shuffle = True)
print('Training dataset size is ', len(train))
print('Training dataset size is ', len(test))

Training dataset size is  5534
Training dataset size is  2372


In [49]:
features = ['km_Driven','City_Code','State_Code','Fuel','Seller_Type','Transmission','Owner','Mileage','Engine','Max_Power','Seats','Region_code']

In [50]:
train_x = train[features]
train_y = train['Selling_Price']
test_x = train[features]
test_y = train['Selling_Price']

In [51]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(train_x, train_y)

In [52]:
prediction = reg.predict(test_x)

In [53]:
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, prediction)
print(np.array(test_y))
print(prediction)

[ 229999  300000  850000 ...  200000  445000 3800000]
[ 203921.42708768  389461.19207593 1011584.77092232 ...  266754.1554756
  411791.71129275 2575847.00525248]


In [54]:
from sklearn import metrics
from sklearn.metrics import r2_score

In [55]:
r2_score(test_y, prediction)

0.6659235215846777